# Generate events

This guide shows how to generate events using Madgraph5 class in HML.

## What is Madgraph5?
[Madgraph5](https://launchpad.net/mg5amcnlo) is a framework providing all the necessary elements for
studies of collider phenomenology. It is able to generate events from a given model in UFO format,
connect to a parton shower tool like Pythia8, and perform detector simulation via tools like Delphes.
It is a powerful and popular tool in the high energy physics community.

Usually, you work with Madgraph5 via its command line interface (CLI). The main commands are
`generate`, `output`, `launch`. `generate` is used to specify the processes you want to study.
`output` is used to decide where to store the output files. `launch` starts generating the events.
After this, you get two prompts: the first one lets you choose extra tools for parton shower and
detector simulation, and the second one lets you tweak the parameters of the current theory model
and generation run. Once you are done with both prompts, the event generation begins.

After some trial and error with different settings, you can make a run script in plain text to
tell Madgraph5 exactly what to do, step by step. It is a good practice to keep this script because
it makes your work reproducible and consistent. It's also beneficial for others to understand what
you have done and repeat your work.

## Use Madgraph5 in HML
While entering commands in the CLI is handy, it can be cumbersome to jump back and forth between
the command line and the file browser. HML makes it easier to access the output data and other
useful information.

To start using it, let's first import the `Madgraph5` class from `hml.generators` module.

In [1]:
from hml.generators import Madgraph5

Welcome to JupyROOT 6.24/02


As you can see, the `Madgraph5` class actually use `PyROOT` as the backend to operate on the `.root`
file. Move on, we'll see how to use it.

In [2]:
signal_generator = Madgraph5(
    executable="mg5_aMC",
    processes="p p > z z, z > j j, z > ve ve~",
    output_dir="./data/pp2zz",
    shower="Pythia8",
    detector="Delphes",
    settings={
        "nevents": 10000,
        "iseed": 42,
        "htjmin": 400,
    },
)
background_generator = Madgraph5(
    executable="mg5_aMC",
    processes="p p > j j / z",
    output_dir="./data/pp2jj",
    shower="Pythia8",
    detector="Delphes",
    settings={
        "nevents": 10000,
        "iseed": 42,
        "htjmin": 400,
    },
)

We presume that our objective is to distinguish Z jets from the usual QCD background, hence we
establish two generators.

The parameters of `Madgraph5` class closely mirror those of the CLI. For example, `define` and `set`
commands in CLI are `definitions` and `settings` here.

Once the generators are configured, we can start generating events with `launch` method.

In [3]:
signal_generator.launch()
background_generator.launch()

Generating events...
Running Pythia8...
Running Delphes...
Storing files...
Done
Generating events...
Running Pythia8...
Running Delphes...
Storing files...
Done


It's worth noting that the `launch` method in the `Madgraph5` class is different from that of the
CLI. It doesn't launch the two prompts, as they are now parts of `Madgraph5` parameters. Instead,
it will directly begin the generation which is more convenient for automation.

After the generation is done, we can access the output files via the `output` attribute.

In [4]:
sig_run = signal_generator.runs[0]
bkg_run = background_generator.runs[0]
print(f"cross section (pb): {sig_run.cross_section}")
print(f"cross section (pb): {bkg_run.cross_section}")
print(f"number of events: {sig_run.events.GetEntries()}")
print(f"number of events: {bkg_run.events.GetEntries()}")

cross section (pb): 0.00077034
cross section (pb): 56849.047629999994
number of events: 10000
number of events: 10000


Each launch in Madgraph5 corresponds to a specific run directory. The `.runs` attribute fetches all
the runs in the output directory intuitively. Accessing cross section is also simple via the
`.cross_section` attribute. To access branches in the `.root` file, use `.events` to read the file
and operate it with `PyROOT` APIs. Here, we use `.GetEntries()` to get the number of events.

To learn more about `Madgraph5` class, check out the [API documentation](../../api-reference/hml.generators#hml.generators.Madgraph5).